In [3]:
import enigma
from importlib import reload

In [185]:
reload(enigma)

<module 'enigma' from '/home/august/phd/enigma/enigma.py'>

In [186]:
message = enigma.Message("augustwollter")
# These parameters describe a rotors initial position, 
# and when the next rotor is advanced
pos = 0
notch = ord('a') - 97
rotorV = enigma.Rotor(pos, notch)

In [214]:
t, s = rotorV.getTransform()
print(enigma.matrix_to_word(t))
s

next rotor is advanced
abcdefghijklmnopqrstuvwxyz


True

1